## Import Statements

In [3]:
import pandas as pd
import numpy as np
from sklearn import model_selection, metrics, preprocessing
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [4]:
# Load in data from csv files
movies_df = pd.read_csv("./Data/ml-latest-small/movies.csv")
ratings_df = pd.read_csv("./Data/ml-latest-small/ratings.csv")

print(f"Movie dataframe dimensions: {movies_df.shape}")
print(f"Ratings dataframe dimensions: {ratings_df.shape}")

# get number of unique users and movies
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())

Movie dataframe dimensions: (9742, 3)
Ratings dataframe dimensions: (100836, 4)


## Test

In [5]:
class Model(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user and item embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.movie_factors = torch.nn.Embedding(n_items, n_factors)
        # fills weights with values from a uniform distribution [0, 0.5]
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.movie_factors.weight.data.uniform_(0, 0.05)
    
    def forward(self, data):
        # matrix multiplication between user and item factors, and then concatenates them to one column
        return (self.user_factors(data[:,0])*self.movie_factors(data[:,1])).sum(1)

In [6]:
class MovieDataset(Dataset):
    def __init__(self, ratings):
        self.ratings = ratings

        le = preprocessing.LabelEncoder()
        le.fit(self.ratings.movieId.values)
        self.lookup = dict(zip(le.transform(self.ratings.movieId.values), self.ratings.movieId.values))

        self.ratings.userId = preprocessing.LabelEncoder().fit_transform(self.ratings.userId.values)
        self.ratings.movieId = preprocessing.LabelEncoder().fit_transform(self.ratings.movieId.values)

        self.x = torch.tensor(self.ratings.drop(['rating', 'timestamp'], axis=1).values)
        self.y = torch.tensor(self.ratings['rating'].values)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, item):
        return (self.x[item], self.y[item])

Train Model

In [7]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = Model(n_users, n_items, n_factors=8)

for name, param in model.named_parameters():
    # prints the parameters who's changes will be recorded
    if param.requires_grad:
        print(name, param.data)

# enable GPU if you have a GPU
if cuda:
    model = model.cuda()

# MSE loss function
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = MovieDataset(ratings_df)
train_loader = DataLoader(train_set, 128, shuffle=True)

for it in range(num_epochs):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

Is running on GPU: False
user_factors.weight tensor([[0.0015, 0.0462, 0.0324,  ..., 0.0005, 0.0394, 0.0298],
        [0.0207, 0.0359, 0.0452,  ..., 0.0371, 0.0230, 0.0445],
        [0.0248, 0.0325, 0.0300,  ..., 0.0028, 0.0499, 0.0222],
        ...,
        [0.0220, 0.0003, 0.0228,  ..., 0.0166, 0.0177, 0.0013],
        [0.0425, 0.0317, 0.0094,  ..., 0.0226, 0.0093, 0.0479],
        [0.0383, 0.0435, 0.0182,  ..., 0.0381, 0.0365, 0.0190]])
movie_factors.weight tensor([[0.0389, 0.0198, 0.0426,  ..., 0.0156, 0.0431, 0.0231],
        [0.0328, 0.0334, 0.0264,  ..., 0.0316, 0.0387, 0.0369],
        [0.0035, 0.0332, 0.0034,  ..., 0.0139, 0.0464, 0.0207],
        ...,
        [0.0360, 0.0096, 0.0336,  ..., 0.0431, 0.0266, 0.0499],
        [0.0012, 0.0092, 0.0028,  ..., 0.0315, 0.0219, 0.0266],
        [0.0248, 0.0386, 0.0230,  ..., 0.0277, 0.0300, 0.0012]])
iter #0 Loss: 11.048663442510033
iter #1 Loss: 4.7379556746047164
iter #2 Loss: 2.474569682391162
iter #3 Loss: 1.7211123744243293
iter #4

In [8]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

user_factors.weight tensor([[ 1.4406,  1.5085,  1.3647,  ...,  1.3247,  1.1655,  1.8264],
        [ 0.9310,  1.1197, -0.0258,  ...,  1.1496,  1.8208,  1.7693],
        [ 1.1364,  1.7194,  0.7117,  ..., -2.6151,  1.3925, -0.2541],
        ...,
        [ 0.3648,  0.5481,  1.3855,  ..., -0.6589,  0.6806,  1.1321],
        [ 0.5329,  1.3394,  1.0249,  ...,  0.6849,  1.2605,  1.1007],
        [ 1.8096,  1.5428,  1.3131,  ...,  0.4340,  0.8188,  0.4304]])
movie_factors.weight tensor([[0.5541, 0.4625, 0.3197,  ..., 0.6063, 0.3152, 0.3232],
        [0.3978, 0.4120, 0.4124,  ..., 0.6047, 0.3821, 0.6489],
        [0.6360, 0.3837, 0.3564,  ..., 0.3592, 0.7338, 0.6272],
        ...,
        [0.4305, 0.4024, 0.4302,  ..., 0.4395, 0.4208, 0.3973],
        [0.4053, 0.4127, 0.4042,  ..., 0.4353, 0.4248, 0.3825],
        [0.4924, 0.5075, 0.4913,  ..., 0.4938, 0.5000, 0.3327]])


In [9]:
movie_names = movies_df.set_index('movieId')['title'].to_dict()
trained_movie_embeddings = model.movie_factors.weight.data.cpu().numpy()
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.lookup[movidx]
    # print(ratings_df.loc[ratings_df['movieId']==movid].count())
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()["userId"]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Go Fish (1994)
	 Chungking Express (Chung Hing sam lam) (1994)
	 On Her Majesty's Secret Service (1969)
	 Three Colors: Blue (Trois couleurs: Bleu) (1993)
	 Bubba Ho-tep (2002)
	 Substitute, The (1996)
	 Insider, The (1999)
	 Money Train (1995)
	 Emma (1996)
	 Death and the Maiden (1994)
Cluster #1
	 Poetic Justice (1993)
	 Lost Weekend, The (1945)
	 Don Juan DeMarco (1995)
	 Being Human (1993)
	 For Whom the Bell Tolls (1943)
	 Bread and Chocolate (Pane e cioccolata) (1973)
	 Kissed (1996)
	 Philadelphia (1993)
	 Inspector General, The (1949)
	 Jefferson in Paris (1995)
Cluster #2
	 Philadelphia Story, The (1940)
	 Afterglow (1997)
	 Unforgiven (1992)
	 Candleshoe (1977)
	 Empire (2002)
	 Blue in the Face (1995)
	 Penny Serenade (1941)
	 Untouchables, The (1987)
	 Battle Royale (Batoru rowaiaru) (2000)
	 Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
Cluster #3
	 Secret of Roan Inish, The (1994)
	 Dangerous Minds (1995)
	 Run Silent Run Deep (1958)
	 National Lampoon's Sen